# Метод опорных векторов

**Метод опорных векторов** (*Support Vector Machine, SVM*) - один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами.

В случае линейно разделимой выборки нужен для максимизации ширины между классами.      
В случае линейно неразделимой выборки просто добавляются штрафы за попадания объекта за полосу.

In [95]:
import pandas as pd
from sklearn.svm import SVC
import numpy as np

In [96]:
data = pd.read_csv('svm-data.csv', header=None)

In [97]:
data.head()

,0,1,2
0,0.0,0.70,0.29
1,1.0,0.23,0.55
2,0.0,0.72,0.42
3,0.0,0.98,0.68
4,0.0,0.48,0.39


In [98]:
# Классификатор - линейный --> ядро - linear
y = data[0]
X = data.loc[:, 1:]
clf = SVC(kernel='linear', C=100000, random_state=241)
clf.fit(X, y)

SVC(C=100000, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [99]:
# индексы опорных объектов
print(clf.support_)

[3 4 9]


In [81]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

## Анализ текстов на основе метода опорных векторов

Можно записывать не количество вхождений слова в текст, а TF-IDF. Это показатель, который равен произведению двух чисел: TF (*term frequency*) и IDF (*inverse document frequency*). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше IDF. Таким образом, TF-IDF будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.

In [82]:
# Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос" и "атеизм"

newsgroups = datasets.fetch_20newsgroups(subset='all', categories=['alt.atheism', 'sci.space'])

In [90]:
# Вычислите TF-IDF-признаки для всех текстов. 

vec = TfidfVectorizer()
X = vec.fit_transform(texts) # признаки
y = newsgroups.target # целевая метка
feature_names = vec.get_feature_names()

In [71]:
# Подберите минимальный лучший параметр C из множества [10^-5, 10^-4, ... 10^4, 10^5] для SVM с линейным ядром
# и параметром random_state=241 при помощи кросс-валидации по 5 блокам.

Cs = {'C': np.power(10.0, np.arange(-5, 6))} # C от 10^-5 до 10^5.
cv = KFold(n_splits=5, shuffle=True, random_state=241)
model = SVC(kernel='linear', random_state=241)

grid_cv = GridSearchCV(model, Cs, scoring='accuracy', cv=cv, n_jobs=-1)
grid_cv.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([  1.00000e-05,   1.00000e-04,   1.00000e-03,   1.00000e-02,
         1.00000e-01,   1.00000e+00,   1.00000e+01,   1.00000e+02,
         1.00000e+03,   1.00000e+04,   1.00000e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [72]:
# Обучите SVM по всей выборке с лучшим параметром C, найденным на предыдущем шаге.

model = grid_cv.best_estimator_
model.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [94]:
# Найдите 10 слов с наибольшим по модулю весом. Они являются ответом на это задание. 
# Укажите их через запятую, в нижнем регистре, в лексикографическом порядке.

weights = np.absolute(model.coef_.toarray()) # absolute = abs
max_weights = sorted(zip(weights[0], feature_names))[-10:]
max_weights.sort(key=lambda x: x[1])

[(1.0293069271856938, 'sci'), (1.0970936466401482, 'keith'), (1.130612344664901, 'bible'), (1.13908083789883, 'religion'), (1.1801315951388633, 'sky'), (1.2016111817520696, 'moon'), (1.2491800073760075, 'atheists'), (1.2546899512384038, 'atheism'), (1.9203794002294938, 'god'), (2.6631647884797105, 'space')]
